### 절대 모멘텀
- 파생변수 'STD-YM' 생성 -> index에서 년-월 추출하여 대입
- 'STD-YM'별 마지막날의 데이터를 month_last_df 저장
- 전월의 기준이 되는 컬럼의 값을 가진 파생변수 생성
- 전년도의 기준이 되는 컬럼의 값을 가진 파생변수 생성
- 전월의 데이터와 전년도의 데이터를 기준으로 거래 내역 생성
- 수익율 계산

In [1]:
from datetime import datetime
import pandas as pd
import numpy as np

In [2]:
# 데이터 로드
df = pd.read_csv('../csv/AAPL.csv', index_col='Date')

In [3]:
df.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1980-12-12,0.513393,0.515625,0.513393,0.513393,0.410525,117258400.0
1980-12-15,0.488839,0.488839,0.486607,0.486607,0.389106,43971200.0
1980-12-16,0.453125,0.453125,0.450893,0.450893,0.360548,26432000.0


In [4]:
# index를 시계열 데이터로 변환
df.index = pd.to_datetime(df.index,utc=True)

In [5]:
type(df.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [6]:
# STD-YM 파생변수 생성해서 인덱스에서 년도, 월 추출해서 저장
# 컬럼의 데이터였다면 -> df['Date'].dt.strftime('%Y-%m')
# 아래는 인덱스 형태로 묶여있음.
df.index.strftime('%Y-%m')

Index(['1980-12', '1980-12', '1980-12', '1980-12', '1980-12', '1980-12',
       '1980-12', '1980-12', '1980-12', '1980-12',
       ...
       '2019-06', '2019-06', '2019-06', '2019-06', '2019-06', '2019-06',
       '2019-06', '2019-06', '2019-06', '2019-06'],
      dtype='object', name='Date', length=9715)

In [7]:
df['STD-YM'] = df.index.strftime('%Y-%m')

In [8]:
df.head(2)

,Open,High,Low,Close,Adj Close,Volume,STD-YM
Date,,,,,,,
1980-12-12 00:00:00+00:00,0.513393,0.515625,0.513393,0.513393,0.410525,117258400.0,1980-12
1980-12-15 00:00:00+00:00,0.488839,0.488839,0.486607,0.486607,0.389106,43971200.0,1980-12


In [9]:
# CASE 1 : 월말의 데이터를 출력하는 방법
# 해당방법은 다중 for문 사용해야함
df.loc['1981-12'].tail(1)
test_df = pd.DataFrame()
for year in range(1980, 2026): #년도 반복
    for month in range (1,13): #월 반복
        try:
            idx = f'{year}-{month}'
            df2 = df.loc[idx].tail(1)
            test_df = pd.concat([test_df,df2],axis=0)
        except Exception as e:
            print(e)
            continue

'1980-1'
'1980-2'
'1980-3'
'1980-4'
'1980-5'
'1980-6'
'1980-7'
'1980-8'
'1980-9'
'1980-10'
'1980-11'
'2019-7'
'2019-8'
'2019-9'
'2019-10'
'2019-11'
'2019-12'
'2020-1'
'2020-2'
'2020-3'
'2020-4'
'2020-5'
'2020-6'
'2020-7'
'2020-8'
'2020-9'
'2020-10'
'2020-11'
'2020-12'
'2021-1'
'2021-2'
'2021-3'
'2021-4'
'2021-5'
'2021-6'
'2021-7'
'2021-8'
'2021-9'
'2021-10'
'2021-11'
'2021-12'
'2022-1'
'2022-2'
'2022-3'
'2022-4'
'2022-5'
'2022-6'
'2022-7'
'2022-8'
'2022-9'
'2022-10'
'2022-11'
'2022-12'
'2023-1'
'2023-2'
'2023-3'
'2023-4'
'2023-5'
'2023-6'
'2023-7'
'2023-8'
'2023-9'
'2023-10'
'2023-11'
'2023-12'
'2024-1'
'2024-2'
'2024-3'
'2024-4'
'2024-5'
'2024-6'
'2024-7'
'2024-8'
'2024-9'
'2024-10'
'2024-11'
'2024-12'
'2025-1'
'2025-2'
'2025-3'
'2025-4'
'2025-5'
'2025-6'
'2025-7'
'2025-8'
'2025-9'
'2025-10'
'2025-11'
'2025-12'


In [10]:
# CASE 2 : 월말의 데이터를 출력하는 방법
# 현재의 STD-YM 데이터와 다음 인덱스의 STD-YM이 다른 경우
flag = df['STD-YM'] != df.shift(-1)['STD-YM']
df.loc[flag]

,Open,High,Low,Close,Adj Close,Volume,STD-YM
Date,,,,,,,
1980-12-31 00:00:00+00:00,0.611607,0.611607,0.609375,0.609375,0.487276,8937600.0,1980-12
1981-01-30 00:00:00+00:00,0.508929,0.508929,0.504464,0.504464,0.403385,11547200.0,1981-01
1981-02-27 00:00:00+00:00,0.473214,0.477679,0.473214,0.473214,0.378397,3690400.0,1981-02
1981-03-31 00:00:00+00:00,0.441964,0.441964,0.437500,0.437500,0.349839,3998400.0,1981-03
1981-04-30 00:00:00+00:00,0.506696,0.511161,0.506696,0.506696,0.405170,3152800.0,1981-04
...,...,...,...,...,...,...,...
2019-02-28 00:00:00+00:00,174.320007,174.910004,172.919998,173.149994,172.485748,28215400.0,2019-02
2019-03-29 00:00:00+00:00,189.830002,190.080002,188.539993,189.949997,189.221313,23564000.0,2019-03
2019-04-30 00:00:00+00:00,203.059998,203.399994,199.110001,200.669998,199.900192,46534900.0,2019-04


In [11]:
# case3
# unique() 함수 이용
ym_list = df['STD-YM'].unique()
month_last_df = pd.DataFrame()
for ym in ym_list:
    data = df.loc[ym].tail(1)
    month_last_df = pd.concat([month_last_df,data],axis=0)
month_last_df

,Open,High,Low,Close,Adj Close,Volume,STD-YM
Date,,,,,,,
1980-12-31 00:00:00+00:00,0.611607,0.611607,0.609375,0.609375,0.487276,8937600.0,1980-12
1981-01-30 00:00:00+00:00,0.508929,0.508929,0.504464,0.504464,0.403385,11547200.0,1981-01
1981-02-27 00:00:00+00:00,0.473214,0.477679,0.473214,0.473214,0.378397,3690400.0,1981-02
1981-03-31 00:00:00+00:00,0.441964,0.441964,0.437500,0.437500,0.349839,3998400.0,1981-03
1981-04-30 00:00:00+00:00,0.506696,0.511161,0.506696,0.506696,0.405170,3152800.0,1981-04
...,...,...,...,...,...,...,...
2019-02-28 00:00:00+00:00,174.320007,174.910004,172.919998,173.149994,172.485748,28215400.0,2019-02
2019-03-29 00:00:00+00:00,189.830002,190.080002,188.539993,189.949997,189.221313,23564000.0,2019-03
2019-04-30 00:00:00+00:00,203.059998,203.399994,199.110001,200.669998,199.900192,46534900.0,2019-04


In [12]:
# month_last_df에서
# 전월의 수정종가 데이터를 BF_1M에 대입
# 전년도의 수정종가 데이터를 BF_12M에 대입
month_last_df['BF_1M'] = month_last_df.shift()['Adj Close']

In [13]:
month_last_df['BF_12M'] = month_last_df.shift(12)['Adj Close']

In [14]:
month_last_df.iloc[10:15]

,Open,High,Low,Close,Adj Close,Volume,STD-YM,BF_1M,BF_12M
Date,,,,,,,,,
1981-10-30 00:00:00+00:00,0.357143,0.359375,0.357143,0.357143,0.285583,13182400.0,1981-10,0.217757,NaN
1981-11-30 00:00:00+00:00,0.334821,0.334821,0.332589,0.332589,0.265949,5992000.0,1981-11,0.285583,NaN
1981-12-31 00:00:00+00:00,0.395089,0.397321,0.395089,0.395089,0.315926,13664000.0,1981-12,0.265949,0.487276
1982-01-29 00:00:00+00:00,0.363839,0.366071,0.363839,0.363839,0.290937,13288800.0,1982-01,0.315926,0.403385
1982-02-26 00:00:00+00:00,0.325893,0.328125,0.325893,0.325893,0.260594,4356800.0,1982-02,0.290937,0.378397


#### 거래 내역 생성
- 모멘텀 인덱스 : (전월의 수정종가 / 전년도의 수정종가) -1
- 모멘텀 인덱스가 0보다는 크고, 무한대가 아닌 경우 -> 매수
* 해당 데이터를 통해 매수 타이밍만 확인.

In [15]:
# 거래내역 df에 생성
df['trade'] = ''

for idx in month_last_df.index:
    signal = ''
    
    # 절대 모멘텀 계산식
    momentum_index = month_last_df.loc[idx,'BF_1M'] / month_last_df.loc[idx,'BF_12M'] - 1
    
    flag = (momentum_index>0) & (momentum_index != np.inf)
    if flag:
        signal='buy'
    
    print(f'날짜 : {idx}, 모멘텀 인덱스 : {momentum_index}, signal : {signal}')
    df.loc[idx:,'trade'] = signal

날짜 : 1980-12-31 00:00:00+00:00, 모멘텀 인덱스 : nan, signal : 
날짜 : 1981-01-30 00:00:00+00:00, 모멘텀 인덱스 : nan, signal : 
날짜 : 1981-02-27 00:00:00+00:00, 모멘텀 인덱스 : nan, signal : 
날짜 : 1981-03-31 00:00:00+00:00, 모멘텀 인덱스 : nan, signal : 
날짜 : 1981-04-30 00:00:00+00:00, 모멘텀 인덱스 : nan, signal : 
날짜 : 1981-05-29 00:00:00+00:00, 모멘텀 인덱스 : nan, signal : 
날짜 : 1981-06-30 00:00:00+00:00, 모멘텀 인덱스 : nan, signal : 
날짜 : 1981-07-31 00:00:00+00:00, 모멘텀 인덱스 : nan, signal : 
날짜 : 1981-08-31 00:00:00+00:00, 모멘텀 인덱스 : nan, signal : 
날짜 : 1981-09-30 00:00:00+00:00, 모멘텀 인덱스 : nan, signal : 
날짜 : 1981-10-30 00:00:00+00:00, 모멘텀 인덱스 : nan, signal : 
날짜 : 1981-11-30 00:00:00+00:00, 모멘텀 인덱스 : nan, signal : 
날짜 : 1981-12-31 00:00:00+00:00, 모멘텀 인덱스 : -0.45421280752591964, signal : 
날짜 : 1982-01-29 00:00:00+00:00, 모멘텀 인덱스 : -0.21681272233722126, signal : 
날짜 : 1982-02-26 00:00:00+00:00, 모멘텀 인덱스 : -0.23113291067318187, signal : 
날짜 : 1982-03-31 00:00:00+00:00, 모멘텀 인덱스 : -0.2551030616940936, signal : 
날짜 : 1982-04-30 00:00

In [16]:
df['trade'].value_counts()

trade
buy    6092
       3623
Name: count, dtype: int64

In [17]:
df['rtn'] = 1

for idx in df.index:
    if df.shift().loc[idx,'trade'] =='' and df.loc[idx,'trade'] =='buy':
        # 매수 가격
        buy = df.loc[idx,'Adj Close']
    elif df.shift().loc[idx,'trade'] =='buy' and df.loc[idx,'trade'] =='':
        # 매도 가격
        sell = df.loc[idx,'Adj Close']
        rtn = sell / buy 
        df.loc[idx,'rtn'] = rtn
        
df['acc_rtn'] = df['rtn'].cumprod()
df[['rtn','acc_rtn']]

C:\Users\Owner\AppData\Local\Temp\ipykernel_33940\2037418445.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.390411905109097' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[idx,'rtn'] = rtn


,rtn,acc_rtn
Date,,
1980-12-12 00:00:00+00:00,1.000000,1.000000
1980-12-15 00:00:00+00:00,1.000000,1.000000
1980-12-16 00:00:00+00:00,1.000000,1.000000
1980-12-17 00:00:00+00:00,1.000000,1.000000
1980-12-18 00:00:00+00:00,1.000000,1.000000
...,...,...
2019-06-18 00:00:00+00:00,1.000000,71.506486
2019-06-19 00:00:00+00:00,1.000000,71.506486
2019-06-20 00:00:00+00:00,1.000000,71.506486


#### 절대 모멘텀 함수 만들기
1. create_ym 함수
    - 매개변수 2개
        - 데이터 프레임 (_df)
        - 기준 컬럼 (_col = 'Adj Close')
    - 데이터 프레임의 복사본을 생성해서 작업
    - 컬럼에 Date가 있다면 Date를 인덱스로 설정
    - 인덱스를 시계열 데이터로 변경
    - 데이터 중 결측치, 양의 무한, 음의 무한이 있으면 제외 flag, ~flag
    - 기준 컬럼 제외하고 모두 삭제
    - 'STD-YM' 파생변수 생성 index에서 '년-월'을 추출하여 대입
    - 위에서 작업된 데이터프레임을 되돌려준다.

In [18]:
def create_ym(_df,
              _col='Adj Close'):
    df = _df.copy()
    
    # 인덱스 변경
    if 'Date' in df.index:
        df.set_index('Date')
    
    # 인덱스 시계열로 변경
    df.index = pd.to_datetime(df.index)
    
    # 데이터 중 결측, 양의 무한, 음의 무한 제외
    flag = df.isin([np.nan,np.inf,-np.inf]).any(axis=1)
    
    # 기준 컬럼 제외하고 삭제
    df = df.loc[~flag,[_col]]
    
    # 'STD-YM' 파생 생성 index에서 년 월 추출 대입
    df['STD-YM'] = df.index.strftime('%Y-%m')
    
    return df

In [19]:
# 데이터 로드
amzn_df = pd.read_csv('../csv/AMZN.csv', index_col='Date')

In [20]:
df_ym = create_ym(amzn_df)

2. 두번째 함수 생성(create_month)
    - 매개변수 5개
        - STD-YM 컬럼이 존재하는 데이터프레임 _df
        - 시작 시간 (_start = '2010-01-01')
        - 종료 시간 (_end = datetime.now())
        - momentum기간 (_momentum = 12)
        - 기준시점[월말shift(-1), 월초shift(1)] :1인 경우는 월말, 0인 경우 월초
    - _select가 1이라면 
        - _df에서 월말의 데이터들을 result에 대입
    - 0이라면
        - _df에서 월 초의 데이터들을 result에 대입
    - 둘 다 아니라면
        - return으로 함수 종료, print를 이용해서 select타입이 잘못됐다 출력
    - 기준이 되는 컬럼의 이름[_df에서 첫번째 컬럼의 이름]을 변수에 저장
    - 전월의 기준이 되는 컬럼의 값을 BF1 컬럼에 대입
    - _momentum 개월 전 데이터를 BF2 컬럼에 대입
    - 시작 시간과 종료 시간을 기준으로 인덱스 필터링
    - 결과(result)를 되돌려준다. : ym_df는 변경이 되지 않은 상태

In [21]:
def create_month(_df,
               _start = '2010-01-01',
               _end = datetime.now(),
               _momentum = 12,
               _select = 1):
    if _select ==1:
        # 월말 데이터 조건식
        flag = _df['STD-YM'] != _df.shift(-1)['STD-YM']
    elif _select == 0:
        # 월초 데이터 조건식
        flag = _df['STD-YM'] != _df.shift(1)['STD-YM']
    else:
        return print('_select의 값은 0 아니면 1 입니다.')
    result = _df.loc[flag]
    
    # 기준이 되는 컬럼의 이름을 변수에 저장
    col = result.columns[0]
    
    # 전월의 기준이 되는 컬럼의 값을 BF1에 대입
    result['BF1'] = result.shift(1)[col]
    # _momentum 값의 과거의 개월 수 데이터  BF2에 대입
    result['BF2'] = result.shift(_momentum)[col]
    try:
        result.index = result.index.dt.tz_localize(None)
    except Exception as e:
        print(e)        
    
    result = result.loc[_start : _end]
    
    
    return result
    
    
        

In [22]:
# warning 메시지 출력 금지
import warnings
warnings.filterwarnings('ignore')

In [23]:
df_ym2 = create_month(df_ym,_select = 1, _momentum=6)

'DatetimeIndex' object has no attribute 'dt'


In [24]:
df_ym2

,Adj Close,STD-YM,BF1,BF2
Date,,,,
2010-01-29,125.410004,2010-01,134.520004,85.760002
2010-02-26,118.400002,2010-02,125.410004,81.190002
2010-03-31,135.770004,2010-03,118.400002,93.360001
2010-04-30,137.100006,2010-04,135.770004,118.809998
2010-05-28,125.459999,2010-05,137.100006,135.910004
...,...,...,...,...
2019-02-28,1639.829956,2019-02,1718.729980,2012.709961
2019-03-29,1780.750000,2019-03,1639.829956,2003.000000
2019-04-30,1926.520020,2019-04,1780.750000,1598.010010


3. 세번째 함수
- 매개변수 3개
    - trade 컬럼이 추가될 데이터 프레임 _df1
    - 월말/ 월초의 데이터 프레임 _df2 -> create_month함수 결과값
    - 모멘텀 스코어 설정(_score = 1)
- _df1의 복사본 생성 (result)
- result에 trade 컬럼을 생성하고 '' 대입
- 반복문을 이용하여 구매내역 추가
    - _df2 데이터프레임 이용하여 momentum_index를 생성
        - BF1 / BF2 - _score
        - result에 있는 trade에 buy 대입
- 구매내역이 추가된 데이터프레임을 되돌려줌

In [25]:
def create_trade(_df1,
                 _df2, 
                 _score=1):
    result = _df1.copy()
    result['trade'] = ''
    
    try:
        result.index = result.index.tz_localize(None)
    except Exception as e:
        print(e)
        
    # 반복문 생성 -> df2를 기준으로 반복
    for idx in _df2.index:
        signal = ''
        
        momentum_index = _df2.loc[idx, 'BF1'] / _df2.loc[idx, 'BF2'] - _score
        
        flag = momentum_index > 0 and momentum_index != np.inf
        if flag:
            signal = 'buy'
        
        result.loc[idx:, 'trade'] = signal
    
    return result


In [26]:
df_ym3 = create_trade(df_ym,df_ym2)

In [27]:
def create_rtn(_df):
    result = _df.copy()
    result['rtn'] = 1

    col = result.columns[0]  # 첫 번째 열을 가격열로 간주
    prev_trade = result['trade'].shift(1)  # 이전 거래 상태 추출

    buy = None  # 매수 가격 저장용

    for idx in result.index:
        if prev_trade.loc[idx] == '' and result.loc[idx, 'trade'] == 'buy':
            buy = result.loc[idx, col]
            print(f'매수일 : {idx}, 매수가 : {buy}')

        elif prev_trade.loc[idx] == 'buy' and result.loc[idx, 'trade'] == '':
            sell = result.loc[idx, col]
            print(f'매도일 : {idx}, 매도가 : {sell}')

            rtn = sell / buy
            result.loc[idx, 'rtn'] = rtn
            print(f'수익률 : {rtn:.4f}')

    result['acc_rtn'] = result['rtn'].cumprod()
    acc_rtn = result['acc_rtn'].iloc[-1]

    return result, acc_rtn


In [28]:
create_rtn(df_ym3)

매수일 : 2010-01-29 00:00:00, 매수가 : 125.410004
매도일 : 2010-06-30 00:00:00, 매도가 : 109.260002
수익률 : 0.8712
매수일 : 2010-10-29 00:00:00, 매수가 : 165.229996
매도일 : 2011-12-30 00:00:00, 매도가 : 173.100006
수익률 : 1.0476
매수일 : 2012-05-31 00:00:00, 매수가 : 212.910004
매도일 : 2014-04-30 00:00:00, 매도가 : 304.130005
수익률 : 1.4284
매수일 : 2014-09-30 00:00:00, 매수가 : 322.440002
매도일 : 2014-11-28 00:00:00, 매도가 : 338.640015
수익률 : 1.0502
매수일 : 2014-12-31 00:00:00, 매수가 : 310.350006
매도일 : 2015-01-30 00:00:00, 매도가 : 354.529999
수익률 : 1.1424
매수일 : 2015-02-27 00:00:00, 매수가 : 380.160004
매도일 : 2016-04-29 00:00:00, 매도가 : 659.590027
수익률 : 1.7350
매수일 : 2016-06-30 00:00:00, 매수가 : 715.619995
매도일 : 2017-01-31 00:00:00, 매도가 : 823.47998
수익률 : 1.1507
매수일 : 2017-02-28 00:00:00, 매수가 : 845.039978
매도일 : 2018-11-30 00:00:00, 매도가 : 1690.170044
수익률 : 2.0001
매수일 : 2019-04-30 00:00:00, 매수가 : 1926.52002


(              Adj Close   STD-YM trade  rtn   acc_rtn
 Date                                                 
 1997-05-15     1.958333  1997-05        1.0  1.000000
 1997-05-16     1.729167  1997-05        1.0  1.000000
 1997-05-19     1.708333  1997-05        1.0  1.000000
 1997-05-20     1.635417  1997-05        1.0  1.000000
 1997-05-21     1.427083  1997-05        1.0  1.000000
 ...                 ...      ...   ...  ...       ...
 2019-06-18  1901.369995  2019-06   buy  1.0  6.246293
 2019-06-19  1908.790039  2019-06   buy  1.0  6.246293
 2019-06-20  1918.189941  2019-06   buy  1.0  6.246293
 2019-06-21  1911.300049  2019-06   buy  1.0  6.246293
 2019-06-24  1907.953857  2019-06   buy  1.0  6.246293
 
 [5563 rows x 5 columns],
 np.float64(6.2462934745238625))